In [37]:
from Fonction_final import main_app
import tkinter as tk
from tkinter import ttk
from tkinter import * 
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from opencage.geocoder import OpenCageGeocode
import nexmo
import winsound
frequency = 2500
duration = 1000
import numpy as np
msg = 'Salut,votre proche s est endormi tout en conduisant.Voici sa localisation'


path = "haarcascade_fronalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_alt.xml')
font = cv2.FONT_HERSHEY_SIMPLEX
key = '042dbd774e464953b4c351b48db871be'
new_model = keras.models.load_model('new_model.h5')
# this is the function called when the button is clicked
def CommencerFunction():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    counter = 0
    smsDetector = 0
    detector = False
    while True:
        ret,frame = cap.read()
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye_tree_eyeglasses.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        eyes = eye_cascade.detectMultiScale(gray,1.1,4)
        for x,y,w,h in eyes:
            roi_gray = gray[y:y+h , x:x+w]
            roi_color = frame[y:y+h , x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2 )
            eyess = eye_cascade.detectMultiScale(roi_gray)
            if len(eyess) == 0:
                print ("eyes are not detected")
            else:
                for (ex,ey,ew,eh) in eyess:
                    eyes_roi = roi_color[ey: ey+eh, ex:ex + ew]
        #Here was the error , it's because you write 244 instead of 224 , because pretrained model use 224pixel * 224pixel images
        #for training so we have to use the same size for validation images and also must be RGB
        final_image =cv2.resize(eyes_roi, (224,224))
        final_image = np.expand_dims(final_image,axis =0)
        final_image = final_image/255.0;
        Predictions = new_model.predict(final_image)
        print(str(Predictions))
        if(Predictions >=  0.1):
            detector = True
            for x,y,w,h in eyes:
                roi_gray = gray[y:y+h , x:x+w]
                roi_color = frame[y:y+h , x:x+w]
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2 )
                eyess = eye_cascade.detectMultiScale(roi_gray)
                for (ex,ey,ew,eh) in eyess:
                    eyes_roi = roi_color[ey: ey+eh, ex:ex + ew]
        #Here was the error , it's because you write 244 instead of 224 , because pretrained model use 224pixel * 224pixel images
        #for training so we have to use the same size for validation images and also must be RGB
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1)
                cv2.putText(frame,"Open eyes" , (x1 + int(w1/10),y1+int(h1/2)),font, 0.7, (0,255,0),2)
        else:
            if detector == False:
                counter = counter + 1
                x1,y1,w1,h1 = 0,0,175,75
                cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1)
                cv2.putText(frame,"Closed Eyes" , (x1 + int(w1/10),y1+int(h1/2)),font, 0.7, (0,0,255),2)
                if counter>5:
                    x1,y1,w1,h1 = 0,0,175,75
                    cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0,0,0),-1)
                    cv2.putText(frame, "Wake up !!", (x1 + int(w1/10),y1+int(h1/2)),font, 0.7, (0,0,255),2)
                    winsound.Beep(frequency, duration)
                    smsDetector = smsDetector + 1
                    counter = 0
                if smsDetector == 3:
                    from ipdata import ipdata
                    from geopy.geocoders import Nominatim
                    ipdata = ipdata.IPData('ea79a526fa149a1e732ca68f12ced279fdff55a4450d2129b0285e2a')
                    lat = ipdata.lookup(select_field='latitude')
                    long = ipdata.lookup(select_field='longitude')
                    geocoder = OpenCageGeocode(key)
                    results = geocoder.reverse_geocode(lat['latitude'],long['longitude'])
                    client = nexmo.Client(key='4fe4ec91', secret='HZz2XXJizXmBX4aI')

                    client.send_message({
                    'from': 'Vonage APIs',
                    'to': '212691946553',
                    'text': msg+'latitude : '+str(lat['latitude'])+' longitude : '+str(long['longitude'])+'city : '+results[0]['components']['region']})
                    smsDetector = 0
            detector = False
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        print(faceCascade.empty())
        faces = faceCascade.detectMultiScale(gray,1.1,4)
        #Draw a rectangle around the face


        for(x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)

        # Use putText() method for
        # inserting text on video
        cv2.imshow('Drowsiness Detection Tutorial', frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break 

    cap.release()
    cv2.destroyAllWindows



# this is the function called when the button is clicked
def btnClickFunction():
	print('clicked')
    
# this is a function to get the user input from the text input box
def getInputBoxValue():
	userInput = Numero.get()
	return userInput


root = Tk()

# This is the section of code which creates the main window
root.geometry('668x436')
root.configure(background='#F0F8FF')
root.title('Hypovigilance Detection Application')

# This is the section of code which creates the a label
Label(root, text='Bienvenue sur', bg='#F0F8FF', font=('arial', 30, 'normal')).place(x=200, y=5)


# This is the section of code which creates the a label
Label(root, text='L\'Application à l\'Hypovigilance', bg='#F0F8FF', font=('arial', 30, 'normal')).place(x=95, y=45)


# This is the section of code which creates the a label
Label(root, text='Chez les Conducteurs', bg='#F0F8FF', font=('arial', 30, 'normal')).place(x=142, y=93)

# First, we create a canvas to put the picture on
photoo= Canvas(root, height=145, width=145)
# Then, we actually create the image file to use (it has to be a *.gif)
picture_file = PhotoImage(file = 'photo.gif')  # <-- you will have to copy-paste the filepath here, for example 'C:\Desktop\pic.gif'
# Finally, we create the image on the canvas and then place it onto the main window
photoo.create_image(145, 0, anchor=NE, image=picture_file)
photoo.place(x=220, y=146)


# This is the section of code which creates a button
Button(root, text='Commencer', bg='#00FF7F', font=('arial', 20, 'bold'), command=CommencerFunction).place(x=8, y=291)


# This is the section of code which creates a button
Button(root, text='Arrêter', bg='#B22222', font=('arial', 20, 'bold'), command=btnClickFunction).place(x=499, y=292)

# This is the section of code which creates the a label
Label(root, text='En Cas D\'urgence : ', bg='#F0F8FF', font=('arial', 15, 'bold')).place(x=7, y=365)

# This is the section of code which creates the a label
Label(root, text='Saisir le numéro à appeler :', bg='#F0F8FF', font=('arial', 18, 'bold')).place(x=86, y=395)


# This is the section of code which creates a text input box
Numero=Entry(root)
Numero.place(x=446, y=405)


root.mainloop()


[[1.]]
False
[[1.]]
False
[[0.500004]]
False
[[0.97561085]]
False
[[0.00470605]]
False
[[0.99997336]]
False
[[1.]]
False
[[0.9999776]]
False
[[1.]]
False
[[1.]]
False
[[0.9974992]]
False
[[1.]]
False
[[1.]]
False
[[0.9994489]]
False
[[1.]]
False
[[1.]]
False
[[0.99999976]]
False
[[1.]]
False
[[0.99144095]]
False
[[1.]]
False
[[1.]]
False
[[0.9998318]]
False
[[1.]]
False
[[1.]]
False
[[0.99999857]]
False
[[1.]]
False
[[1.]]
False
[[0.9995751]]
False
[[0.99999714]]
False
[[1.]]
False
[[1.]]
False
[[0.9959633]]
False
[[1.]]
False
[[1.]]
False
[[1.]]
False
[[1.]]
False
[[1.]]
False
[[1.]]
False
[[0.99998564]]
False
[[0.9988471]]
False
[[1.]]
False
[[0.16859388]]
False
[[0.7948234]]
False
[[1.]]
False
[[1.]]
False
[[0.00375223]]
False
eyes are not detected
eyes are not detected
[[0.00375223]]
False
eyes are not detected
[[1.1005306e-12]]
False
eyes are not detected
[[1.1005306e-12]]
False
eyes are not detected
eyes are not detected
[[1.1005306e-12]]
False
eyes are not detected
[[1.1005306e-

<ipython-input-37-a65cd882edfe>:94: DeprecationWarning: Call to deprecated method send_message. (nexmo.Client#send_message is deprecated. Use Sms#send_message instead)
  client.send_message({


False
eyes are not detected
[[3.1037805e-13]]
False
eyes are not detected
[[5.7668026e-10]]
False
eyes are not detected
[[5.7668026e-10]]
False
[[7.434561e-08]]
False
eyes are not detected
[[6.3056325e-09]]
False
eyes are not detected
[[3.7084863e-10]]
False
eyes are not detected
[[2.0114771e-12]]
False
eyes are not detected
eyes are not detected
[[2.0114771e-12]]
False
eyes are not detected
eyes are not detected
[[2.0114771e-12]]
False
eyes are not detected
[[7.2678636e-13]]
False
eyes are not detected
[[2.4690655e-10]]
False
eyes are not detected
[[1.879727e-11]]
False
eyes are not detected
eyes are not detected
[[1.879727e-11]]
False
[[2.7799887e-09]]
False
eyes are not detected
eyes are not detected
[[2.7799887e-09]]
False
eyes are not detected
[[2.2361302e-11]]
False
eyes are not detected
[[1.453188e-12]]
False
[[1.2436884e-13]]
False
[[0.04412961]]
False
[[3.104687e-05]]
False
[[0.05268329]]
False
[[0.9007133]]
False
[[0.14198709]]
False
[[0.00821382]]
False
[[0.9999527]]
False
[